Importing Libraries

In [33]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split,DataLoader
import torch.optim as optim



Loading Datasets

In [34]:
# setting file path
train_filepath = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
test_filepath = "../input/asl-alphabet/asl_alphabet_test"

In [35]:
# defining transforms
train_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [36]:
#Data Loading 
train_dataset = datasets.ImageFolder(train_filepath, transform=train_transforms)
print("Train Dataset Info:\n")
train_dataset

#### DataLoader
DataLoader takes a dataset (such as you would get from ImageFolder) and returns batches of images and the corresponding labels.

In [37]:
# Splitting dataset for traing and validation
# torch.manual_seed(1)
# indices = torch.randperm(len(train_dataset))

# split = int(len(train_dataset) * 0.2)

# train_data = torch.utils.data.Subset(train_dataset, indices[split:])
# val_data = torch.utils.data.Subset(train_dataset, indices[:split])

# print("Number of images in train dataset : \t",len(train_data))
# print("Number of images in validation dataset :",len(val_data))

In [38]:

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=100, 
                                               shuffle=True,
                                               num_workers=4)

In [39]:
train_label = train_dataloader.dataset.classes
print("Labels :\t",train_label,"\nLength of the label list :",len(train_label))

In [40]:
# for i in range(4):
#     for img, label in train_dataloader:
#         print('Ground truth', train_label[label[0]])
#         plt.imshow(img[0].permute(1, 2, 0))
#         plt.show()
#         break

In [41]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device
# inputs, labels = inputs.to(device), labels.to(device)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# # inputs, labels = inputs.to(device), labels.to(device)
# device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data, labels = data.to(device), labels.to(device)

In [42]:
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.NN = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2)
        )

        self.predict = nn.Sequential(
            nn.Linear(16*100*100,100),
            nn.Dropout(0.2),
            nn.BatchNorm1d(100),
            nn.Dropout(0.2),
            nn.Linear(100,29),
            nn.Softmax(dim=1)
        )
    
    def forward(self,x):
        x = self.NN(x)
        x = x.view(-1,16*100*100)
        return self.predict(x)


In [43]:
#creating a model 
model = NN()

if torch.cuda.is_available():
    model.cuda()
    
model

In [44]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.01)

In [47]:
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    for batch_idx , (data,labels) in enumerate(train_dataloader):
        data = data.to(device=device)
        labels = labels.to(device= device)
        data, labels = data.cuda(), labels.cuda()
        
        #forward
        scores = model(data)
        loss = criterion(scores, labels)

        #backprop
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()

        #performance
        prediction = torch.max(scores,1)[1]
    
        correct += (prediction == labels).cpu().sum().numpy()

    print("Epoch :",epoch,"\tLoss :",loss.data,"\tAccuracy :",(correct/len(train_dataloader.dataset))*100)        

In [49]:
model1 = NN()

if torch.cuda.is_available():
    model1.cuda()
    
model1

In [50]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.01, weight_decay = 0.0001)

In [51]:
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    for batch_idx , (data,labels) in enumerate(train_dataloader):
        data = data.to(device=device)
        labels = labels.to(device= device)
        data, labels = data.cuda(), labels.cuda()
        
        #forward
        scores = model(data)
        loss = criterion(scores, labels)

        #backprop
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()

        #performance
        prediction = torch.max(scores,1)[1]
    
        correct += (prediction == labels).cpu().sum().numpy()

    print("Epoch :",epoch,"\tLoss :",loss.data,"\tAccuracy :",(correct/len(train_dataloader.dataset))*100)   